<a href="https://colab.research.google.com/github/OIJTUMUCT/MFTI/blob/main/Lebedev_IA_DL_CAPTCHA.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split
from tensorflow.keras.utils import to_categorical
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras import layers, models

In [ ]:
# загрузка
images = np.load('data/images.npy')
labels = np.load('data/labels.npy')
images_sub = np.load('data/images_sub.npy')

In [ ]:
# нормализация
images = images.astype('float32') / 255.0
images_sub = images_sub.astype('float32') / 255.0

In [ ]:
labels_cat = to_categorical(labels, num_classes=26)
X_train, X_val, y_train, y_val = train_test_split(images, labels_cat, test_size=0.1, random_state=42)

In [ ]:
# аугментация
train_gen = ImageDataGenerator(
    rotation_range=15,
    width_shift_range=0.1,
    height_shift_range=0.1,
    shear_range=0.1,
    zoom_range=0.1,
    horizontal_flip=False
).flow(X_train, y_train, batch_size=64)

In [ ]:
model = models.Sequential([
    # 1-ый C-B-C-B-P-D
    layers.Conv2D(32, (3, 3), activation='relu', padding='same', input_shape=(48, 48, 3)),
    layers.BatchNormalization(),
    layers.Conv2D(32, (3, 3), activation='relu', padding='same'),
    layers.BatchNormalization(),
    layers.MaxPooling2D((2, 2)),
    layers.Dropout(0.25),

    # 2-ой C-B-C-B-P-D
    layers.Conv2D(64, (3, 3), activation='relu', padding='same'),
    layers.BatchNormalization(),
    layers.Conv2D(64, (3, 3), activation='relu', padding='same'),
    layers.BatchNormalization(),
    layers.MaxPooling2D((2, 2)),
    layers.Dropout(0.3),

    # 3-ий C-B-C-B-P-D
    layers.Conv2D(128, (3, 3), activation='relu', padding='same'),
    layers.BatchNormalization(),
    layers.Conv2D(128, (3, 3), activation='relu', padding='same'),
    layers.BatchNormalization(),
    layers.MaxPooling2D((2, 2)),
    layers.Dropout(0.35),

    # C-B-C-B-G
    layers.Conv2D(256, (3, 3), activation='relu', padding='same'),
    layers.BatchNormalization(),
    layers.Conv2D(256, (3, 3), activation='relu', padding='same'),
    layers.BatchNormalization(),
    layers.GlobalAveragePooling2D(),

    # Dense-Drop-Dense
    layers.Dense(256, activation='relu'),
    layers.Dropout(0.5),
    layers.Dense(26, activation='softmax')
])

/opt/homebrew/Caskroom/miniconda/base/envs/tf-mac/lib/python3.11/site-packages/keras/src/layers/convolutional/base_conv.py:107: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)
2025-05-23 20:17:12.871253: I metal_plugin/src/device/metal_device.cc:1154] Metal device set to: Apple M4 Max
2025-05-23 20:17:12.871289: I metal_plugin/src/device/metal_device.cc:296] systemMemory: 48.00 GB
2025-05-23 20:17:12.871294: I metal_plugin/src/device/metal_device.cc:313] maxCacheSize: 18.00 GB
2025-05-23 20:17:12.871315: I tensorflow/core/common_runtime/pluggable_device/pluggable_device_factory.cc:305] Could not identify NUMA node of platform GPU ID 0, defaulting to 0. Your kernel may not have been built with NUMA support.
2025-05-23 20:17:12.871326: I tensorflow/core/common_runtime/pluggable_devic

In [ ]:
import tensorflow as tf

class EarlyStoppingByAccuracy(tf.keras.callbacks.Callback): # ранняя остановка обучения по accuracy
    def __init__(self, target=0.985):
        super().__init__()
        self.target = target

    def on_epoch_end(self, epoch, logs=None):
        logs = logs or {}
        val_acc = logs.get('val_accuracy')
        if val_acc is not None and val_acc >= self.target:
            print(f"\nДостигнута целевая точность {val_acc:.4f} ≥ {self.target:.4f}, обучение остановлено.")
            self.model.stop_training = True

In [ ]:
import plotly.graph_objects as go
from tensorflow.keras.callbacks import Callback
from IPython.display import display, clear_output

class LiveAccuracyPlot(Callback): # построение графика в процессе обучения (обновление каждую эпоху)
    def __init__(self):
        super().__init__()
        self.train_acc = []
        self.val_acc = []
        self.epochs = []

    def on_epoch_end(self, epoch, logs=None):
        logs = logs or {}
        self.epochs.append(epoch)
        self.train_acc.append(logs.get('accuracy', 0))
        self.val_acc.append(logs.get('val_accuracy', 0))

        # отрисовка интерактивного графика
        fig = go.Figure()
        fig.add_trace(go.Scatter(x=self.epochs, y=self.train_acc,
                                 mode='lines+markers', name='Train Accuracy'))
        fig.add_trace(go.Scatter(x=self.epochs, y=self.val_acc,
                                 mode='lines+markers', name='Validation Accuracy'))

        fig.update_layout(
            title='Accuracy Over Epochs',
            xaxis_title='Epoch',
            yaxis_title='Accuracy',
            xaxis=dict(rangeslider=dict(visible=True)),
            yaxis=dict(range=[0, 1]),
            height=400,
            margin=dict(l=40, r=40, t=40, b=40)
        )

        clear_output(wait=True)
        display(fig)

In [ ]:
model.compile(
    optimizer='AdamW',
    loss=tf.keras.losses.CategoricalCrossentropy(label_smoothing=0.1),
    metrics=['accuracy']
) # компилирование

In [ ]:
target_accuracy = 0.975
history = model.fit(
    train_gen,
    epochs=200, # количество эпох
    batch_size=128,
    validation_data=(X_val, y_val),
    steps_per_epoch=len(X_train) // 64,
    callbacks=[
        EarlyStoppingByAccuracy(target=target_accuracy),
        LiveAccuracyPlot()
    ],
    verbose=1
) # обучение

281/281 ━━━━━━━━━━━━━━━━━━━━ 0s 626us/step - accuracy: 0.8906 - loss: 1.2705 - val_accuracy: 0.9610 - val_loss: 1.2669


In [ ]:
# предсказание
preds = model.predict(images_sub)
classes = np.argmax(preds, axis=1)

# сохранение результатов
submission = pd.DataFrame({'Id': np.arange(len(classes)), 'Category': classes})
submission.to_csv('submission.csv', index=False)

1563/1563 ━━━━━━━━━━━━━━━━━━━━ 8s 5ms/step
